# NetworkProject ?? good title

> Network Analysis on USA Airports
- toc: true
- branch: master
- badges: false
- comments: true
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Purpose

The purpose of this project is to gain insights from a *network analysis* of USA airports. This network consists of passenger flights between airports in the United States that occurred during December of 2010.

## Dataset

The data comes from the Research and Innovative Technology Administration (RITA). See http://www.rita.dot.gov/about_rita/ for details. The airport position information was collected from Wikipedia (by Gabor) and other public online sources.

The dataset is a directed network with edge directions aligning with flight directions. The network contains multiple edges between many vertices. Each edge is associated with a single carrier aircraft type. Multiple carriers between the same pair of airports will have multiple edges between the vertices for the airports. This kind of network is sometimes called a *multigraph*. In total the network has 755 vertices and 23,473 edges.

The analysis was performed in R using the igraphdata package, written by Gabor Csardi (Gabor, 2015). He also maintains the package. The package is available in the CRAN repository (https://cran.r-project.org/web/packages/igraphdata/igraphdata.pdf). This package consists of a collection of network datasets to use with the igraph package. Examples of these datasets are: The Enron email network, various food webs, interactions in the immunoglobulin protein, the karate club network, Koenigsberg's bridges, visuotactile brain areas of the macaque monkey, UK faculty friendship network, and the domestic US flights network.

I will start by working with the complete dataset. The dataset will be slimmed down progressively until I arrive at a suitable subset for my purposes. I have a general interest in the operations of airlines. In addition, the network seemed rich in terms of the number of attributes and the multiplicity of edges.

Next, I will describe the attributes included with the network. The network has a ‘name’ graph attribute. It also has several vertex and edge attributes.

### Vertex attributes

- **name:** Symbolic vertex name, this is the three letter IATA airport code.
- **City:** City and state, where the airport is located.
- **Position:** Position of the airport, in WGS coordinates.

### Edge attributes

- **Carrier:** Name of the airline. The network includes both domestic and international carriers that performed at least one flight in December of 2010.
- **Departures:** The number of departures (for a given airline and aircraft type.
- **Seats:** The total number of seats available on the flights carried out by a given airline, using a given aircraft type.
- **Passengers:** The total number of passengers on the flights carried out by a given airline, using a given aircraft type.
- **Aircraft:** Type of the aircraft.
- **Distance:** The distance between the two airports, in miles.

## Visualization and Network Summary

Although the terms “node” and “link” (rather than *vertex* and *edge*) are more digestible to the general reader, I have decided, for the sake of preciseness, to use the more technical *vertex* and *edge*. This usage will be extended to the visualizations as well.

A summary of this network in R confirms that it is *directed* and *named*. It is not *bipartite*; neither is it *weighted*. There are, however, edge attributes that can be used to make it weighted.

```
> summary(usa)
IGRAPH bf6202d DN-- 755 23473 -- US airports
+ attr: name (g/c), name (v/c), City (v/c), Position (v/c), Carrier (e/c), Departures (e/n),
| Seats (e/n), Passengers (e/n), Aircraft (e/n), Distance (e/n)![image.png](attachment:image.png)
```

Figure 1 shows a visualization of the network. Note that the size of vertices has been made proportional to the log of the degree due to the large variation of the degree of vertices. The symbolic vertex names (airport codes) are not shown to improve clarity. There are 755 vertices and 23,473 edges (although this is not apparent from the visualization). It is quite clear that, as we might expect, that this is a *scale-free* (rather than a random) network. A number of prominent, highly connected vertices are easy to spot.  Another characteristic of this network is its apparent partitioning of vertices into a number of cohesive subgroups. There seems to be two main subgroups and several smaller ones. There are even small “islands” of a few connected vertices. The visualization reveals the presence of some *self-loops* in the network.

![Figure 1  Network with vertex size proportional to the log of the number of connected edges](../images/net_fig1.png "Figure 1  Network with vertex size proportional to the log of the number of connected edges")

The R-code finds 53 self-loops:
```
> sum(which_loop(usa))
[1] 53
```

Figure 2 shows the network again, this time without taking the log of the degree. This provides some additional insights.

![Figure 2  Network with vertex size proportional to the number of connected edges](../images/net_fig2.png "Figure 2  Network with vertex size proportional to the number of connected edges")

Figure 3 is the same as Figure 2 except that the layout of the vertices is along a circle. This visualization emphasizes how highly-connected the hub vertices are in this scale-free network.

![Figure 3  Network with vertex size proportional to the number of connected edges](../images/net_fig3.png "Figure 3  Network with vertex size proportional to the number of connected edges")

## Community Detection

We can approach the presence of cohesion in a network from a number of angles. One approach is to consider *cliques*.

### Cliques

A clique is a maximally complete subgraph, in other words, a subset of vertices that have all possible ties among them. Cliques are usually considered in the context of undirected networks. It is, however, somewhat interesting to pretend for a moment that our network is undirected and consider the presence of cliques.

The R code reveals that there are cliques present and that the size of the largest maximally complete subgraph is 27:
```
> clique.number(usa)
[1] 27
```

To proceed toward proper community detection, we will remove all the self-loops in the network. The occurrence of these are somewhat abnormal. They could be data errors or simply the fact that an airplane had to turn back to its takeoff airport due to problems that were experienced. As mentioned above, there are 53 self-loops in the network.

After removal, we have this network:
```
> summary(usa2)
IGRAPH b11c16a DN-- 755 23420 -- US airports
+ attr: name (g/c), name (v/c), City (v/c), Position (v/c), Carrier (e/c), Departures (e/n),
| Seats (e/n), Passengers (e/n), Aircraft (e/n), Distance (e/n)
```

Next, I will simplify the network by collapsing all the multi-edges. In other words, I will turn the multigraph into a simple graph. Doing this will simplify the analysis. It does not make sense to combine the values for *Carrier* and *Aircraft* from multiple edges into a single value for my purposes. I will ignore these edge attributes. For the other edge attributes I will combine them by summing. This means I will sum all the *Departures*, *Seats*, and *Passengers* for all the similarly-directed edges between a given vertex pair. The network will still be directional. The *Distance* edge attribute values will be combined by taking the max of sets of equal values. After simplification, we have:
```
> summary(usa3)
IGRAPH ddbc561 DN-- 755 8228 -- US airports
+ attr: name (g/c), name (v/c), City (v/c), Position (v/c), Departures (e/n), Seats (e/n),
| Passengers (e/n), Distance (e/n)
> is_simple(usa3) #confirm that usa3 is now a simple graph
[1] TRUE
```

Next, I will turn the graph into a *weighted* network. I will use the *Distance* edge attribute to weight each edge.  After weighting, we have:
```
> summary(usa4)
IGRAPH ddbc561 DNW- 755 8228 -- US airports
+ attr: name (g/c), name (v/c), City (v/c), Position (v/c), Departures (e/n), Seats (e/n),
| Passengers (e/n), Distance (e/n), weight (e/n)
```

## REFERENCES

Berinato, S. (2016). Good Charts - The HBR Guide to Making Smarter, More Persuasive Data Visualizations.

Tufte, E. R. (1990). Envisioning Information.

Tufte, E. R. (2001). The Visual Display of Quantitative Information.

Tufte, E. R. (2006). Beautiful Evidence.